### converting mbox files to csv.

In [ ]:

import mailbox
import pandas as pd
import re
# Path to your .mbox file
mbox_path = '../Mbox Files/Amine_All_mails.mbox'

# Path where you want to save the CSV file
csv_path = '../Output CSV/Raw Email/amine_emails.csv'

# Define the email fields you want to extract
fields = ['date_parsed', 'from', 'to', 'subject', 'body']

# Function to extract the body of an email
def get_email_body(message):
    body = None
    if message.is_multipart():
        for part in message.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True)  # decode
                break
    else:
        body = message.get_payload(decode=True)
    return body.decode('utf-8', errors='ignore') if body else None

# Open the .mbox file
mbox = mailbox.mbox(mbox_path)

# Initialize a list to hold all email data
emails = []

# Iterate through each message in the .mbox file
for message in mbox:
    email_data = {
        'date': message['date'],
        'from': message['from'],
        'to': message.get('to', 'N/A'),  # Handle cases where 'to' might be missing
        'subject': message['subject'],
        'body': get_email_body(message)
    }
    emails.append(email_data)

# Convert the list of emails to a pandas DataFrame
df_emails = pd.DataFrame(emails)

# Save the DataFrame to a CSV file
df_emails.to_csv(csv_path, index=False, encoding='utf-8')

print("Conversion completed.")

### Features Engineering Functions

In [ ]:

import pandas as pd
import re
from dateutil.parser import parse    

def load_data(filepath):
    """Load data from a CSV file."""
    return pd.read_csv(filepath)

def preprocess_data(df):
    """Preprocess data by parsing dates and setting appropriate data types."""
    # Apply the function to the date column
    df['str_body'] = df['body'].astype(str).replace({'None': '', 'nan': '', 'NaT': ''})
    df['date_parsed'] = df['date'].apply(clean_and_parse_email_date)
    df['date_parsed_pd'] = pd.to_datetime(df['date_parsed'],utc=True)
    df['date_year'] = df['date_parsed_pd'].dt.year
    df['date_year'] = df['date_year'].fillna(0).astype(int).astype(str)  # Fill NaN, convert to int, then to string
    df['date_year'] = df['date_year'].replace('0', '')
    df['date_quarter'] = df['date_parsed_pd'].dt.quarter.fillna(0).astype(int).astype(str).apply(lambda x: 'Q' + x)
    df['date_YYYY_QQ'] = df['date_year'] + '-' + df['date_quarter']
    df['date_YYYY_MM'] = df['date_parsed_pd'].dt.strftime('%Y-%m')
    df['date_month_nb'] = df['date_parsed_pd'].dt.month
    df['date_month_name'] = df['date_parsed_pd'].dt.month_name()    
    df['date_day'] = df['date_parsed_pd'].dt.day
    df['day_of_week_name'] = df['date_parsed_pd'].dt.day_name()
    df['isweekend'] = df['date_parsed_pd'].dt.dayofweek >= 5
    df['date_hour'] = df['date_parsed_pd'].dt.strftime('%Y-%m-%d %H:00')
    df['time_of_day'] = df['date_parsed_pd'].dt.hour.apply(categorize_time)
    df[['from', 'to', 'subject', 'body']] = df[['from', 'to', 'subject', 'body']].astype('string')
    df['from'] = df['from'].fillna('Unknown Sender')
    df['to'] = df['to'].fillna('Unknown Recipient')
    df['clean_from'] = df['from'].apply(extract_email)
    df['clean_to'] = df['to'].apply(extract_email)

    return df

def clean_and_parse_email_date(date_str):
    """Removing potential trailing descriptors like "(GMT) and converting to timestemps"""
    cleaned_date = date_str.split('(')[0].strip()
    try:
        # Parsing the cleaned date string into a datetime object
        return pd.to_datetime(cleaned_date, format='%a, %d %b %Y %H:%M:%S %z', errors='coerce')
    except ValueError:
        # Fallback to fuzzy parsing if the strict format fails
        return parse(cleaned_date, fuzzy=True, ignoretz=True)

def categorize_time(hour):
    if 0 <= hour <= 8:  # Night category now includes early morning
        return 'Night'
    elif 9 <= hour <= 17:  # Work-day during typical business hours
        return 'Work-day'
    elif 18 <= hour <= 23:  # Evening hours
        return 'Evening'

def extract_email(email):
    """Extract the email address using regex, return in lowercase."""
    match = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', email)
    return match.group(0).lower() if match else email.lower()

def filter_emails_that_contain_sku_in_title(df, keyword='EV'):
    """Filter emails by subject containing a specific keyword."""
    return df[df['subject'].str.contains(keyword, case=False, na=False)]

def features_creation(df):
    """Orchestrate the creation of features by calling specific functions."""
    df = extract_sku_and_connections(df)
    df = compute_rankings(df)
    df = calculate_date_differences(df)
    
    df['constant_value'] = 1
    df['row_id'] = df.index + 1  # Using +1 to make it 1-based index
    return df

def add_company_names(df, company_filepath):
    """
    Enhance the DataFrame with company names associated with email addresses.

    Parameters:
    - df (pd.DataFrame): The main DataFrame containing 'clean_to' and 'clean_from' columns.
    - company_filepath (str): The file path to the CSV containing company names and email associations.

    Returns:
    - pd.DataFrame: The enhanced DataFrame with 'company_name_to' and 'company_name_from' added.
    """
    # Read the extra data containing company names associated with emails
    company_name_and_email_df = pd.read_csv(company_filepath)

    # Merge to add 'company_name_from'
    df = pd.merge(df, company_name_and_email_df[['email', 'Company Name']], left_on='clean_from', right_on='email', how='left')
    df.rename(columns={'Company Name': 'company_name_from'}, inplace=True)
    df.drop(columns=['email'], inplace=True)  # Remove the redundant 'email' column after merge

    # Merge to add 'company_name_to'
    df = pd.merge(df, company_name_and_email_df[['email', 'Company Name']], left_on='clean_to', right_on='email', how='left')
    df.rename(columns={'Company Name': 'company_name_to'}, inplace=True)
    df.drop(columns=['email'], inplace=True)  # Remove the redundant 'email' column after merge
    
    df[['company_name_from', 'company_name_to']] = df[['company_name_from', 'company_name_to']].astype('string')

    return df

def extract_sku_and_connections(df):
    """Extract SKU value and normalize email connections."""
    df['SKU'] = df['subject'].str.extract('(EV-[A-Z0-9-]+)')
    df.loc[:, 'email_connection'] = df.apply(lambda x: format_connection(x['clean_from'], x['clean_to']), axis=1)
    df.loc[:, 'company_connection'] = df.apply(lambda x: format_connection(x['company_name_to'], x['company_name_from']), axis=1)
    return df

def format_connection(from_field, to_field):
    """Format connections by sorting and normalizing case."""
    connected_field = sorted([from_field, to_field])
    return ' - '.join(connected_field)

def compute_rankings(df):
    """Compute various rankings."""
    df['email_rank_by_sku_pair'] = df.groupby(['SKU', 'company_connection'])['date_parsed_pd'].rank(method='first')
    df['email_rank_by_sku'] = df.groupby(['SKU', 'company_connection'])['email_rank_by_sku_pair'].transform('max')
    df['max_rank_by_sku_pair'] = df.groupby('SKU')['date_parsed_pd'].rank(method='first')
    df['is_max_rank_equal_to_sku_pair_rank'] = df['email_rank_by_sku_pair'] == df['email_rank_by_sku']
    return df

def calculate_date_differences(df):
    """Calculate the date of the first email and the number of days from the first email for each group."""
    df['first_email_pair_date'] = df.groupby(['SKU', 'company_connection'])['date_parsed_pd'].transform('min')
    df['days_from_first_email'] = (df['date_parsed_pd'] - df['first_email_pair_date']).dt.days
    return df

def is_internal_communication(connection):
    """
    Checks if the communication is internal based on exact domain matching for 'bus.com'.

    Args:
    connection (str): A string containing two email addresses separated by ' - '.

    Returns:
    int: Returns 1 if both emails are from 'bus.com', 0 otherwise.
    """
    emails = connection.split(' - ')
    # Regex to match exactly 'bus.com' as the domain part of the email
    pattern = r'[\w\.-]+@bus\.com\b'
    
    # Check both emails against the pattern
    is_first_internal = re.fullmatch(pattern, emails[0].strip())
    is_second_internal = re.fullmatch(pattern, emails[1].strip()) if len(emails) > 1 else False

    # Return 1 if both are internal, 0 otherwise
    return int(is_first_internal and is_second_internal)

def select_emails(df, specific_skus):
    if specific_skus is None or not specific_skus:
        raise ValueError("At least one SKU must be provided.")
    if isinstance(specific_skus, str):
        specific_skus = [specific_skus]
    return df[df['SKU'].isin(specific_skus)]

def sample_skus(df, n=30, random_state=1):
    """
    Randomly sample SKUs from a DataFrame.

    Args:
    df (pd.DataFrame): DataFrame containing SKUs.
    n (int, optional): Number of SKUs to sample. Defaults to 30.
    random_state (int, optional): Seed for the random number generator to ensure reproducibility.

    Returns:
    list: List of sampled SKUs.
    """
    unique_skus = df['SKU'].dropna().unique()
    if len(unique_skus) < n:
        # If there are fewer unique SKUs than n, return all unique SKUs.
        return unique_skus.tolist()
    else:
        # Randomly sample n SKUs from the unique SKU list.
        return df['SKU'].dropna().sample(n, random_state=random_state).tolist()

def reorder_id_columns(df):
    columns = list(df.columns)
    columns.insert(0, columns.pop(columns.index('row_id')))
    df = df[columns]
    return df

def export_emails(df, filepath='sample_amine_emails.csv'):
    """Export DataFrame to CSV."""
    df.to_csv(filepath, index=False)
    print(f"CSV file '{filepath}' has been created with the filtered and sorted email data.")
    
# Function to extract price from email body
def extract_price(body):
    # Use regex to find price patterns, assuming prices are formatted as $ followed by numbers
    pattern = r"(?:\w+\s){0,30}\d+[\d,]*\s*\$[\w\s]*"
    matches = re.findall(pattern, body)
    return matches[0] if matches else None



# Data Processing

In [ ]:
raw_csv_filepath = '../Output CSV/Raw Email/amine_emails.csv'
prepared_csv_filepath = '../Output CSV/Prepared CSV/test_prepared_amine_emails.csv'
price_test_csv_filepath = '../Output CSV/Prepared CSV/price_test_amine_emails.csv'
company_csv_filepath = '../Extra Data/amine_unique_emails_company_association.csv'

df = load_data(raw_csv_filepath)
df_preprocess = preprocess_data(df)
df_sku_only_emails = filter_emails_that_contain_sku_in_title(df_preprocess)
df_with_company_names = add_company_names(df_sku_only_emails, company_csv_filepath)
df_with_all_features = features_creation(df_with_company_names)
df_reorder_column = reorder_id_columns(df_with_all_features)

# sampling data
# list_sample_sku = sample_skus(df_reorder_column, n=30, random_state=1)
# df_sample = df_reorder_column[df_reorder_column['SKU'].isin(list_sample_sku)]

# test_skus = ['EV-AC-578901']  # Example list of specific SKUs
test_skus = ["EV-AC-578901", "EV-JL-558646", "EV-NA-459859", "EV-NA-511353", "EV-NA-518377"]
df_test_sku = select_emails(df_reorder_column, test_skus)

In [ ]:
# df_sample_with_test_sku = pd.concat([df_sample, df_test_sku]).drop_duplicates()

df_test_sku_subset = df_test_sku[['row_id','str_body','SKU','company_connection'
                                  ,'email_rank_by_sku_pair'
                                  ,'email_rank_by_sku'
                                  ,'is_max_rank_equal_to_sku_pair_rank'
                                  ,'is_internal_communication'
                                  ,'has_no_response'
                                  ,'is_not_available']]

df_filtered = df_test_sku_subset[df_test_sku_subset['is_max_rank_equal_to_sku_pair_rank'] == True]

# add price raw data
# need to clean-up for it to work?
df_filtered['re_price_info'] = df_filtered['str_body'].apply(extract_price)
df_filtered['has_price_info'] = df_filtered['re_price_info'].apply(lambda x: 1 if x is not None else 0)

export_emails(df_filtered, filepath=price_test_csv_filepath)
#print(df_sample_with_test_sku.shape)
#df_sample_with_test_sku.head()


# Email Analysis

In [ ]:
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

# SQL query for task 1: Count of emails and SKUs by quarter, compute avg emails/SKU
query1 = """
SELECT 
    date_YYYY_QQ,
    COUNT(row_id) AS email_count,
    COUNT(DISTINCT SKU) AS unique_skus,
    CAST(ROUND(COUNT(row_id) / COUNT(DISTINCT SKU),0) AS INTEGER) AS avg_emails_per_sku
FROM 
    df_reorder_column
GROUP BY 
    date_YYYY_QQ;
"""

result1 = pysqldf(query1)
result1.head()

In [ ]:
query = """
SELECT 
    clean_to AS recipient,
    company_name_to AS company,
    COUNT(row_id) AS emails_sent,
    SUM(CASE WHEN date_YYYY_QQ = '2022-Q4' THEN 1 ELSE 0 END) AS Q4_2022,
    SUM(CASE WHEN date_YYYY_QQ = '2023-Q1' THEN 1 ELSE 0 END) AS Q1_2023,
    SUM(CASE WHEN date_YYYY_QQ = '2023-Q2' THEN 1 ELSE 0 END) AS Q2_2023,
    SUM(CASE WHEN date_YYYY_QQ = '2023-Q3' THEN 1 ELSE 0 END) AS Q3_2023,
    SUM(CASE WHEN date_YYYY_QQ = '2023-Q4' THEN 1 ELSE 0 END) AS Q4_2023,
    SUM(CASE WHEN date_YYYY_QQ = '2024-Q1' THEN 1 ELSE 0 END) AS Q1_2024
FROM 
    df_reorder_column
WHERE 
    clean_from = 'amine.bedaida@bus.com'
GROUP BY 
    clean_to, company
ORDER BY 
    emails_sent DESC;
"""

# Execute the query
result = pysqldf(query)
result.head(20)

In [ ]:
query = """
SELECT 
    SKU,
    COUNT(*) AS count_emails,
    (
        SELECT COUNT(DISTINCT company) 
        FROM (
            SELECT company_name_from AS company FROM df_reorder_column sub WHERE sub.SKU = main.SKU AND company_name_from != 'bus.com'
            UNION
            SELECT company_name_to AS company FROM df_reorder_column sub WHERE sub.SKU = main.SKU AND company_name_to != 'bus.com'
        )
    ) AS count_unique_companies,
    MIN(date_parsed_pd) AS first_email_date,
    MAX(date_parsed_pd) AS last_email_date
FROM 
    df_reorder_column main
WHERE 
    SKU IS NOT NULL 
GROUP BY 
    SKU
ORDER BY 
    count_emails DESC;
"""

# Execute the query
result = pysqldf(query)

result['first_email_date'] = pd.to_datetime(result['first_email_date']).dt.date
result['last_email_date'] = pd.to_datetime(result['last_email_date']).dt.date

result.head()

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Determine the bin edges and midpoints
max_value = result['count_unique_companies'].max()
bins = range(1, max_value + 2)  # Bins from 1 to max + 1
bin_centers = [0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)]  # Calculate midpoints for x-ticks

# Plotting the histogram
plt.figure(figsize=(8, 6))
sns.histplot(result['count_unique_companies'], bins=bins, kde=False, color='green')
plt.title('Distribution Companies Reached Out Per SKU')
plt.xlabel('# Companies')
plt.ylabel('# SKU')
plt.xticks(bin_centers, labels=range(1, max_value + 1))  # Set custom x-ticks at bin centers
plt.grid(True)
plt.show()

# Regex provide bad result 

In [ ]:
# Function to extract price from email body
def extract_price(body):
    # Use regex to find price patterns, assuming prices are formatted as $ followed by numbers
    pattern = r"(?:\w+\s){0,30}\d+[\d,]*\s*\$[\w\s]*"
    matches = re.findall(pattern, body)
    return matches[0] if matches else None

# Apply the extract_price function to get prices from the email body
df_test_sku['price_info'] = df_test_sku['str_body'].apply(extract_price)

# now try pivot tables 
pivot_table = df_test_sku.pivot_table(index='SKU', columns='company_connection'
                                            , values='price_info'
                                            , aggfunc='first'
                                            , fill_value='NA')
pivot_table.head()


# Anonymize email to leverage Aganta platform for prompt engineering

In [ ]:
# anonimized emails dataset
test_price_csv_path = '../Test/Data/sample_train_price_amine_emails.csv'
df_test_price = pd.read_csv(test_price_csv_path)
#email_sample = df_test_price['str_body'].iloc[0]
# smaller_email = email_sample[0:100]
#print(email_sample)
# len(email_sample)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                  api_key="")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert email anonymizer"),
    ("human", (
        "Replace all email addresses, physical adresses, location, company names, website, first names, last names, full names, names, phone numbers, dates (ex: August 4, 2023), job title, and timestamps "
        "(ex: 10:30am) in this email below (Between \"### Start email###\" and \"###End email###\") "
        "with the following string \"[anonymous]\"\n"
        "### Start email###\n"
        "{email_body}\n"
        "###End email###"
    ))
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

#print(chain.invoke({"email_body": email_sample}))

In [ ]:
def anonimized_email(email_body):
    result = chain.invoke({"email_body": email_body})
    return result

# Apply the function to the 'str_body' column and create a new column with the results
df_test_price['ano_email'] = df_test_price['str_body'].apply(anonimized_email)

In [ ]:
# export the files. Check if anonomized work
csv_file_path = '../Test/Data/test_price_amine_ano_emails.csv'
df_test_price.to_csv(csv_file_path, index=False)